In [6]:
from datascience import Table
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns; sns.set()
from sklearn import ensemble
from sklearn import model_selection
from sklearn import tree
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import GridSearchCV
import graphviz
import xgboost as xgb
from xgboost import plot_importance
%matplotlib inline

**Read Data**

In [7]:
data = pd.read_csv('engineered.csv')

In [14]:
features_x = [
    'pickup_hour_sin',
    'pickup_hour_cos',
    'pickup_day_sin',
    'pickup_day_cos',
    'pickup_week_sin',
    'pickup_week_cos',
    'pickup_month_sin',
    'pickup_month_cos',
    'pickup_loc_x',
    'pickup_loc_y',
    'pickup_loc_z',
    'dropoff_loc_x',
    'dropoff_loc_y',
    'dropoff_loc_z',
    'passenger_count_2',
    'passenger_count_3',
    'passenger_count_4',
    'passenger_count_5',
    'passenger_count_6'
]
features_y = ['trip_duration']

In [15]:
X = data[features_x]
Y = data[features_y]

**Split Into Train & Test**

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=123)

In [18]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear',
                          colsample_bytree = 0.3,
                          learning_rate = 0.1,
                          max_depth = 5,
                          alpha = 10,
                          gamma = 0,
                          min_child_weight = 1,
                          n_estimators = 100)

In [ ]:
def modelfit(alg, dtrain, predictors, useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds, show_progress=False)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['Disbursed'],eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print "\nModel Report"
    print "Accuracy : %.4g" % metrics.accuracy_score(dtrain['Disbursed'].values, dtrain_predictions)
    print "AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['Disbursed'], dtrain_predpro)

**Hyperparamater Tuning**

In [ ]:
xg_reg.fit(X_train,y_train)